# Création fichier csv clean

Ce notebook est consacré à la création du fichier qui sera utilisé comme base de données. Pour cela, nou savons pris le format CSV, avec comme séparateur la barre verticale "|" 
Il est séparé en plusieurs étapes : 

- Installations et importations
- Fonctions utilitaires
- Script permettant l'écriture du fichier
    

### Installations et importations

Installations pour la récupération des données


In [1]:
#Installations pour la récupération des données

%pip install webdriver-manager
%pip install -q lxml
%pip install bs4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Installations de spacy et du package des mots français inutiles pour le cleaning des questions

In [2]:
#Installation pour le cleaning

%pip install spacy 
%python -m spacy download fr_core_news_sm

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


Import des modules nécessaires

In [3]:
#imports

import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

import spacy

nlp = spacy.load('fr_core_news_sm')

### Fonctions utilitaires

Plusieurs fonctions utilitaires sont définies ici : celle qui permet d'obtenir le lien d'une page, pour plus de lisibilité, et celle qui permet de nettoyer le texte contenu dans une question.

In [4]:
#Fonctions utilitaires

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"


def clean_question(text, nom, remove_stopwords=True):
    
    text = nlp(text)

    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(nom.lower().split()+nom.split()+['m.','mme','ministre','été', 'cas',"redemande","interroge", "demande", "luire", "année", "faire", "nombre","ailleurs","dernier","concerner","situation","dispositif","aujourd'hui","part","pouvoir","permettre",'ans',"jour","appeller", "attention", "attirer", "non", "pourtant"]))
    
    tokens = [tok.lemma_ for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean



### Création du fichier

On crée d'abord le fichier, en précisant l'encodage pour éviter les problemes d'accents. Ensuite, on applique le script de scraping que l'on a pu déjà réalise dans le notebook [`Scraping Dataframe`](https://github.com/jcomperatore/PythonPourLaDataScience/blob/main/Datascrapping/Scraping%20Dataframe.ipynb), auquel on rajoute l'étape de nettoyage, a la ligne 34.

Pour éviter de modifier le fichier final, tous les "data_cleaned.csv" seront remplacés par des "data_cleaned_TEST.csv" pour le lecteur.

In [5]:
#création fichier

données = open("data_cleaned_TEST.csv", "w", encoding="utf-8")

données.write("id|groupe|nom|ministère_interrogé|rubrique|titre|date|question|question_clean")

données.close()

Pour tester ce code (assez long),  il est recomandé d'essayer pour un petit nombre de questions (une cinquantaine prend une minute, raisonnable)

In [6]:
données = open("data_cleaned_TEST.csv", "a", encoding="utf-8")

for k in range(1, 51) : 
    
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    rubrique0 = page.findAll("div", {"class" : "question_col33"})
    rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
    rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]
    
    ministere0 = page.find("div", {"class" : "question_col50"})
    ministere = ministere0.text.strip()[23:]

    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    date0 = page.find("div", {"class" : "question_publish_date"})
    date = date0.find("span")
    date = date.text.strip()
    
    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    question_clean = clean_question(question, nom)

    données.write("\n" + str(k) + "|" + groupe + "|" + nom +  "|" + ministere + "|" + rubrique + "|" + titre + "|" + date + "|" + question + "|" + question_clean)

données.close()